#imports


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error
df= pd.read_csv ('/content/drive/MyDrive/projeto_insufiencia_cardiaca/heart_tratado.csv')

In [ ]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289.0,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180.0,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283.0,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214.0,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195.0,0,Normal,122,N,0.0,Up,0


In [ ]:
df.shape


(917, 12)

In [ ]:
df2=pd.DataFrame.copy(df)

In [ ]:
df2 ['Sex'].replace({'M':1,'F':0},inplace=True)
df2 ['ChestPainType'].replace({'TA':0, 'ATA':1, 'NAP':2, 'ASY':3},inplace=True)
df2 ['RestingECG'].replace({'Normal':0, 'ST':1, 'LVH':2},inplace=True)
df2 ['ExerciseAngina'].replace({'Y':1, 'N':0},inplace=True)
df2 ['ST_Slope'].replace({'Up':0, 'Flat':1, 'Down':2},inplace=True)


In [ ]:
df2.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,1,1,140,289.0,0,0,172,0,0.0,0,0
1,49,0,2,160,180.0,0,0,156,0,1.0,1,1
2,37,1,1,130,283.0,0,1,98,0,0.0,0,0
3,48,0,3,138,214.0,0,0,108,1,1.5,1,1
4,54,1,2,150,195.0,0,0,122,0,0.0,0,0


In [ ]:
previsores= df2.iloc[0:11].values
previsores

array([[ 40. ,   1. ,   1. , 140. , 289. ,   0. ,   0. , 172. ,   0. ,
          0. ,   0. ,   0. ],
       [ 49. ,   0. ,   2. , 160. , 180. ,   0. ,   0. , 156. ,   0. ,
          1. ,   1. ,   1. ],
       [ 37. ,   1. ,   1. , 130. , 283. ,   0. ,   1. ,  98. ,   0. ,
          0. ,   0. ,   0. ],
       [ 48. ,   0. ,   3. , 138. , 214. ,   0. ,   0. , 108. ,   1. ,
          1.5,   1. ,   1. ],
       [ 54. ,   1. ,   2. , 150. , 195. ,   0. ,   0. , 122. ,   0. ,
          0. ,   0. ,   0. ],
       [ 39. ,   1. ,   2. , 120. , 339. ,   0. ,   0. , 170. ,   0. ,
          0. ,   0. ,   0. ],
       [ 45. ,   0. ,   1. , 130. , 237. ,   0. ,   0. , 170. ,   0. ,
          0. ,   0. ,   0. ],
       [ 54. ,   1. ,   1. , 110. , 208. ,   0. ,   0. , 142. ,   0. ,
          0. ,   0. ,   0. ],
       [ 37. ,   1. ,   3. , 140. , 207. ,   0. ,   0. , 130. ,   1. ,
          1.5,   1. ,   1. ],
       [ 48. ,   0. ,   1. , 120. , 284. ,   0. ,   0. , 120. ,   0. ,
          0. ,   0. 

In [ ]:
alvo=df2.iloc[:,11].values
alvo

array([0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
df2.describe()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
count,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.00000,917.000000
mean,53.509269,0.789531,2.251908,132.540894,244.635389,0.233370,0.604144,136.789531,0.404580,0.886696,0.63795,0.552890
std,9.437636,0.407864,0.931502,17.999749,53.347125,0.423206,0.806161,25.467129,0.491078,1.066960,0.60727,0.497466
min,28.000000,0.000000,0.000000,80.000000,85.000000,0.000000,0.000000,60.000000,0.000000,-2.600000,0.00000,0.000000
25%,47.000000,1.000000,2.000000,120.000000,214.000000,0.000000,0.000000,120.000000,0.000000,0.000000,0.00000,0.000000
50%,54.000000,1.000000,3.000000,130.000000,244.635389,0.000000,0.000000,138.000000,0.000000,0.600000,1.00000,1.000000
75%,60.000000,1.000000,3.000000,140.000000,267.000000,0.000000,1.000000,156.000000,1.000000,1.500000,1.00000,1.000000
max,77.000000,1.000000,3.000000,200.000000,603.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.00000,1.000000


In [ ]:
std=StandardScaler()
previsores_esc=std.fit_transform(previsores)

In [ ]:
previsores_esc_df=pd.DataFrame(previsores_esc)
previsores_esc_df.describe()


,0,1,2,3,4,5,6,7,8,9,10,11
count,1.100000e+01,1.100000e+01,1.100000e+01,1.100000e+01,1.100000e+01,11.0,11.000000,1.100000e+01,1.100000e+01,1.100000e+01,1.100000e+01,1.100000e+01
mean,-5.248327e-16,1.211152e-16,-5.046468e-18,-9.436896e-16,1.917658e-16,0.0,0.000000,-1.034526e-16,-4.037175e-17,-4.037175e-17,4.037175e-17,4.037175e-17
std,1.048809e+00,1.048809e+00,1.048809e+00,1.048809e+00,1.048809e+00,0.0,1.048809,1.048809e+00,1.048809e+00,1.048809e+00,1.048809e+00,1.048809e+00
min,-1.158325e+00,-1.095445e+00,-9.701425e-01,-1.725376e+00,-1.265693e+00,0.0,-0.316228,-1.666094e+00,-4.714045e-01,-5.996254e-01,-6.123724e-01,-6.123724e-01
25%,-1.001022e+00,-1.095445e+00,-9.701425e-01,-6.219379e-01,-6.916720e-01,0.0,-0.316228,-7.335237e-01,-4.714045e-01,-5.996254e-01,-6.123724e-01,-6.123724e-01
50%,1.001022e-01,9.128709e-01,3.638034e-01,-2.541252e-01,-5.559942e-01,0.0,-0.316228,1.179536e-01,-4.714045e-01,-5.996254e-01,-6.123724e-01,-6.123724e-01
75%,6.506643e-01,9.128709e-01,3.638034e-01,4.815003e-01,8.947143e-01,0.0,-0.316228,9.694309e-01,-4.714045e-01,2.248595e-01,5.103104e-01,5.103104e-01
max,1.515833e+00,9.128709e-01,1.697749e+00,1.952751e+00,2.053194e+00,0.0,3.162278,1.334350e+00,2.121320e+00,1.873829e+00,1.632993e+00,1.632993e+00


In [ ]:
previsores2 = df.iloc[:,0:11].values
previsores2

array([[40, 'M', 'ATA', ..., 'N', 0.0, 'Up'],
       [49, 'F', 'NAP', ..., 'N', 1.0, 'Flat'],
       [37, 'M', 'ATA', ..., 'N', 0.0, 'Up'],
       ...,
       [57, 'M', 'ASY', ..., 'Y', 1.2, 'Flat'],
       [57, 'F', 'ATA', ..., 'N', 0.0, 'Flat'],
       [38, 'M', 'NAP', ..., 'N', 0.0, 'Up']], dtype=object)

In [ ]:
previsores2 [:,1]=LabelEncoder().fit_transform(previsores2[:,1])
previsores2 [:,2]=LabelEncoder().fit_transform(previsores2[:,1])
previsores2 [:,6]=LabelEncoder().fit_transform(previsores2[:,6])
previsores2 [:,8]=LabelEncoder().fit_transform(previsores2[:,8])
previsores2 [:,10]=LabelEncoder().fit_transform(previsores2[:,10])
previsores2

array([[40, 1, 1, ..., 0, 0.0, 2],
       [49, 0, 0, ..., 0, 1.0, 1],
       [37, 1, 1, ..., 0, 0.0, 2],
       ...,
       [57, 1, 1, ..., 1, 1.2, 1],
       [57, 0, 0, ..., 0, 0.0, 1],
       [38, 1, 1, ..., 0, 0.0, 2]], dtype=object)

In [ ]:
previsores3 = ColumnTransformer (transformers=[('OneHot',OneHotEncoder(),[1,2,6,8,10])],remainder='passthrough').fit_transform(previsores2)

In [ ]:
previsores3df=pd.DataFrame(previsores3)
previsores3df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,40,140,289.0,0,172,0.0
1,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,49,160,180.0,0,156,1.0
2,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,37,130,283.0,0,98,0.0
3,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,48,138,214.0,0,108,1.5
4,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,54,150,195.0,0,122,0.0


In [ ]:
previsores3_esc=std.fit_transform(previsores3)
previsores3_esc=pd.DataFrame(previsores3_esc)
previsores3_esc.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,-0.516309,0.516309,-0.516309,0.516309,-0.507826,0.815013,-0.490781,0.824310,-0.824310,-0.271607,-1.001091,1.149573,-1.432206,0.414627,0.832075,-0.551733,1.383339,-0.831502
1,1.936826,-1.936826,1.936826,-1.936826,-0.507826,0.815013,-0.490781,0.824310,-0.824310,-0.271607,0.998910,-0.869888,-0.478057,1.526360,-1.212261,-0.551733,0.754736,0.106251
2,-0.516309,0.516309,-0.516309,0.516309,-0.507826,-1.226974,2.037569,0.824310,-0.824310,-0.271607,-1.001091,1.149573,-1.750256,-0.141240,0.719543,-0.551733,-1.523953,-0.831502
3,1.936826,-1.936826,1.936826,-1.936826,-0.507826,0.815013,-0.490781,-1.213136,1.213136,-0.271607,0.998910,-0.869888,-0.584074,0.303453,-0.574578,-0.551733,-1.131075,0.575128
4,-0.516309,0.516309,-0.516309,0.516309,-0.507826,0.815013,-0.490781,0.824310,-0.824310,-0.271607,-1.001091,1.149573,0.052026,0.970493,-0.930931,-0.551733,-0.581047,-0.831502


# pos pre prossecessameno


In [ ]:
x_train,x_test,y_train,y_test=train_test_split(previsores2, alvo, test_size=0.3, random_state=0)

In [ ]:
x_train.shape

(641, 11)

In [ ]:
MLPClassifier?

CALCULANDO NUMERO DE CAMADAS ESCONDIDA

11+1/2=6

PARA ESSE DF VOU UTILIZAR 6 HIDDEN-LAYER


In [ ]:
rede= MLPClassifier(hidden_layer_sizes=(6), activation='identity', solver='adam', random_state=3,learning_rate= 'adaptive' ,tol=0.000001, max_fun=200000, max_iter=1000,verbose=True)
rede.fit(x_train,y_train)

Iteration 1, loss = 19.79308495
Iteration 2, loss = 19.79308494
Iteration 3, loss = 19.77910679
Iteration 4, loss = 19.71669724
Iteration 5, loss = 19.59994099
Iteration 6, loss = 19.17831507
Iteration 7, loss = 18.04955603
Iteration 8, loss = 15.94783445
Iteration 9, loss = 13.04158873
Iteration 10, loss = 9.74929140
Iteration 11, loss = 6.77573539
Iteration 12, loss = 4.77201125
Iteration 13, loss = 3.85440212
Iteration 14, loss = 3.77512562
Iteration 15, loss = 3.93959061
Iteration 16, loss = 4.04699518
Iteration 17, loss = 4.01044473
Iteration 18, loss = 3.87321356
Iteration 19, loss = 3.71300804
Iteration 20, loss = 3.58677607
Iteration 21, loss = 3.51744026
Iteration 22, loss = 3.48984749
Iteration 23, loss = 3.48478565
Iteration 24, loss = 3.46679734
Iteration 25, loss = 3.42482894
Iteration 26, loss = 3.37447707
Iteration 27, loss = 3.31951584
Iteration 28, loss = 3.27727007
Iteration 29, loss = 3.23902749
Iteration 30, loss = 3.20150877
Iteration 31, loss = 3.16563813
Iteratio

MLPClassifier(activation='identity', hidden_layer_sizes=6,
              learning_rate='adaptive', max_fun=200000, max_iter=1000,
              random_state=3, tol=1e-06, verbose=True)

In [ ]:
rede.score(x_test,y_test)

0.8514492753623188

In [ ]:
previsoes=rede.predict(x_test)
previsoes

array([1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0])

In [ ]:
print("acuracia: %.2f%%" % (accuracy_score(y_test, previsoes) * 100))

acuracia: 85.14%


In [ ]:
kfold=KFold(n_splits=10, shuffle=True, random_state=10)
modelo= MLPClassifier (hidden_layer_sizes=(6), activation='identity', solver='adam', random_state=3,learning_rate= 'adaptive' ,tol=0.000001, max_fun=200000, max_iter=1000,verbose=True)
resultados=cross_val_score(modelo, previsores2, alvo, cv=kfold)


Iteration 1, loss = 19.83493389
Iteration 2, loss = 19.83066181
Iteration 3, loss = 19.80884846
Iteration 4, loss = 19.63116983
Iteration 5, loss = 19.00530008
Iteration 6, loss = 17.15986300
Iteration 7, loss = 14.04735108
Iteration 8, loss = 10.17027763
Iteration 9, loss = 6.66627431
Iteration 10, loss = 4.54436698
Iteration 11, loss = 3.85237732
Iteration 12, loss = 4.00259061
Iteration 13, loss = 4.12356236
Iteration 14, loss = 4.05046511
Iteration 15, loss = 3.85650647
Iteration 16, loss = 3.69757057
Iteration 17, loss = 3.63475092
Iteration 18, loss = 3.64721798
Iteration 19, loss = 3.61220763
Iteration 20, loss = 3.54241056
Iteration 21, loss = 3.46487641
Iteration 22, loss = 3.40477729
Iteration 23, loss = 3.35074014
Iteration 24, loss = 3.29044742
Iteration 25, loss = 3.24297515
Iteration 26, loss = 3.19694219
Iteration 27, loss = 3.16198034
Iteration 28, loss = 3.10391221
Iteration 29, loss = 3.06562359
Iteration 30, loss = 3.01672484
Iteration 31, loss = 2.97444657
Iteration

In [ ]:
resultado = cross_val_score(modelo, previsores2, alvo, cv=kfold)
print("Acurácia: %.2f%%" % (resultado.mean()*100))

Iteration 1, loss = 14.57892999
Iteration 2, loss = 13.67745671
Iteration 3, loss = 12.41904834
Iteration 4, loss = 10.82418496
Iteration 5, loss = 9.19263690
Iteration 6, loss = 7.67692589
Iteration 7, loss = 6.54339440
Iteration 8, loss = 5.97910865
Iteration 9, loss = 5.86748931
Iteration 10, loss = 5.98042142
Iteration 11, loss = 6.07362620
Iteration 12, loss = 5.99970071
Iteration 13, loss = 5.83444627
Iteration 14, loss = 5.67052863
Iteration 15, loss = 5.58336794
Iteration 16, loss = 5.47843003
Iteration 17, loss = 5.41439130
Iteration 18, loss = 5.36124415
Iteration 19, loss = 5.30301441
Iteration 20, loss = 5.24430032
Iteration 21, loss = 5.18796507
Iteration 22, loss = 5.12442558
Iteration 23, loss = 5.07228304
Iteration 24, loss = 5.00829788
Iteration 25, loss = 4.95282843
Iteration 26, loss = 4.89494619
Iteration 27, loss = 4.82510093
Iteration 28, loss = 4.75699264
Iteration 29, loss = 4.67455180
Iteration 30, loss = 4.60119431
Iteration 31, loss = 4.54614536
Iteration 32,

#regressão linear com redes neurais artificiais


In [ ]:
dados= pd.read_csv('/content/drive/MyDrive/cursoudemyminiciante/housing.csv')
dados.head()

,RM,LSTAT,PTRATIO,MEDV
0,6.575,4.98,15.3,504000.0
1,6.421,9.14,17.8,453600.0
2,7.185,4.03,17.8,728700.0
3,6.998,2.94,18.7,701400.0
4,7.147,5.33,18.7,760200.0


In [ ]:
dados.shape

(489, 4)

In [ ]:
independentes=dados.iloc[:,0:3].values
independentes

array([[ 6.575,  4.98 , 15.3  ],
       [ 6.421,  9.14 , 17.8  ],
       [ 7.185,  4.03 , 17.8  ],
       ...,
       [ 6.976,  5.64 , 21.   ],
       [ 6.794,  6.48 , 21.   ],
       [ 6.03 ,  7.88 , 21.   ]])

In [ ]:
dependentes=dados.iloc[:,3].values
dependentes

array([ 504000.,  453600.,  728700.,  701400.,  760200.,  602700.,
        480900.,  569100.,  346500.,  396900.,  315000.,  396900.,
        455700.,  428400.,  382200.,  417900.,  485100.,  367500.,
        424200.,  382200.,  285600.,  411600.,  319200.,  304500.,
        327600.,  291900.,  348600.,  310800.,  386400.,  441000.,
        266700.,  304500.,  277200.,  275100.,  283500.,  396900.,
        420000.,  441000.,  518700.,  646800.,  732900.,  558600.,
        531300.,  518700.,  445200.,  405300.,  420000.,  348600.,
        302400.,  407400.,  413700.,  430500.,  525000.,  491400.,
        396900.,  743400.,  518700.,  663600.,  489300.,  411600.,
        392700.,  336000.,  466200.,  525000.,  693000.,  493500.,
        407400.,  462000.,  365400.,  438900.,  508200.,  455700.,
        478800.,  491400.,  506100.,  449400.,  420000.,  436800.,
        445200.,  426300.,  588000.,  501900.,  520800.,  480900.,
        501900.,  558600.,  472500.,  466200.,  495600.,  6027

In [ ]:
dependentes.shape, independentes.shape


((489,), (489, 3))

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(independentes, dependentes, test_size=0.3, random_state=0)

In [ ]:
x_train.shape, x_test.shape

((342, 3), (147, 3))

In [ ]:
MLPRegressor?

In [ ]:
ml= MLPRegressor(hidden_layer_sizes=(100,100), activation='relu', solver='lbfgs', random_state=12, max_iter=5000,verbose=False)

In [ ]:
ml.fit(x_train,y_train)

MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=5000, random_state=12,
             solver='lbfgs')

In [ ]:
ml.n_layers_

4

In [ ]:
ml.score(x_train,y_train)

0.8536909521067109

In [ ]:
previsoes_ml = ml.predict(x_test)
previsoes_ml

array([ 407767.5422182 ,  701972.97746861,  360968.61260667,
        281563.70376671,  504260.72534338,  294965.6791673 ,
        359661.83736238,  431768.94901103,  442911.66636384,
        397832.99417084,  260141.63435007,  371667.00215915,
        454351.69865628,  226556.5690648 ,  507897.48862296,
        337577.04366966,  443883.56705347,  581355.47934491,
        379434.56248093,  662706.21063195,  656462.36051081,
        707732.77231626,  233167.74937699,  422449.31968087,
        395208.70855643,  894121.71653521,  785687.31679089,
        889086.76826785,  429798.37739983,  432212.51644925,
        304791.7724169 ,  410770.49971032,  454159.07587872,
        717608.9934564 ,  433960.36992074, 1004099.59438876,
        475574.50404788,  253389.73178383,  485572.05145804,
        548798.09174937,  265174.97141844,  377724.48636809,
        423111.58269052,  478653.31057874,  295901.29030843,
        390240.67345036,  501059.85130627,  324797.84885648,
        910002.13737627,

In [ ]:
mean_absolute_error (y_test, previsoes_ml)

54889.90145604481

In [ ]:
np.sqrt(mean_absolute_error (y_test, previsoes_ml))

234.28593951845426

In [ ]:
kfold=KFold(n_splits=12, shuffle=True, random_state=5)
modelo= MLPRegressor(hidden_layer_sizes=(100,100), activation='relu', solver='lbfgs', random_state=12,learning_rate= 'adaptive', max_iter=5000,verbose=False)
resultados=cross_val_score(modelo, independentes, dependentes, cv=kfold)
resultados

array([0.90323151, 0.87189736, 0.81416127, 0.78567585, 0.80046946,
       0.80653329, 0.56896982, 0.79833847, 0.74399243, 0.68651471,
       0.84949644, 0.5445834 ])

In [ ]:
print("Acurácia: %.2f%%" % (resultados.mean()*100))

Acurácia: 76.45%


#escalonamento

In [223]:
xscl= StandardScaler()
x_train_esc=xscl.fit_transform(x_train)
x_test_esc=xscl.transform(x_test)

In [224]:
x_test_esc

array([[-6.24507256e-01, -6.20687880e-01,  1.17917695e+00],
       [ 9.56985082e-01, -8.43295235e-01, -2.61727885e+00],
       [-1.30072075e+00,  1.98112421e+00, -1.81053199e+00],
       [ 9.72674490e-01,  9.82935809e-01,  7.99531373e-01],
       [ 2.90185237e-01, -5.72479917e-01, -3.39405369e-01],
       [ 2.72926888e-01,  9.46070896e-01,  7.99531373e-01],
       [-2.08519115e+00,  2.33134087e+00, -1.81053199e+00],
       [-1.88341711e-01, -2.51777567e-02,  7.99531373e-01],
       [-1.08325729e-01, -2.13755962e-01, -2.44493974e-01],
       [-4.18976010e-01,  1.39296468e-01,  1.17917695e+00],
       [-1.41892805e-02,  1.26651206e+00,  7.99531373e-01],
       [-6.19800434e-01,  4.03022379e-01,  7.99531373e-01],
       [-2.99736508e-01, -7.29864736e-01,  5.14797187e-01],
       [-2.51664988e+00,  3.05162455e+00,  7.99531373e-01],
       [ 7.76556889e-01, -4.85989161e-01,  1.13172126e+00],
       [-7.45315699e-01,  6.32719141e-01,  1.27408835e+00],
       [ 7.36714050e-02, -1.24429444e-01

In [225]:
x_train_esc

array([[ 0.05327517, -0.70150711, -0.05467118],
       [ 1.12799963, -0.44487061, -0.52922816],
       [ 0.60711128, -0.79792304,  0.230063  ],
       ...,
       [-0.33111532, -0.36121561, -0.33940537],
       [-0.31699486,  0.84398345, -0.29194967],
       [-0.33268427, -0.38815536, -0.90887374]])

In [296]:
yscl=StandardScaler()
y_train_esc=yscl.fit_transform(y_train.reshape(-1,1))
y_test_esc=yscl.transform(y_test.reshape(-1,1))

In [322]:
redes= MLPRegressor(hidden_layer_sizes=(6,6,6), activation='relu', solver='lbfgs', random_state=12, max_iter=5000,verbose=False)

In [323]:
redes.fit(x_train_esc,y_train_esc.ravel())

MLPRegressor(hidden_layer_sizes=(6, 6, 6), max_iter=5000, random_state=12,
             solver='lbfgs')

In [324]:
redes.n_layers_

5

In [325]:
redes.score(x_train_esc,y_train_esc)

0.8820723711015378

In [326]:
redes.score(x_test_esc,y_test_esc)

0.8303270280287622